# 🧠 FOA Intelligence: Pipeline Demo

Welcome to the interactive demo of the **FOA Intelligence Pipeline**. In this notebook, we will walk through the core stages of our AI-powered funding intelligence system:

1. **Ingestion**: Fetching FOAs from public APIs (Grants.gov & NSF).
2. **Extraction**: Mapping raw source data to a standardized schema.
3. **Semantic Tagging**: Applying ontology-based tags using NLP.
4. **Export**: Saving processed records to JSON and CSV.

---

In [1]:
import sys
from pathlib import Path

# Add root to path so we can import our source code
sys.path.append(str(Path.cwd()))

import pandas as pd
from IPython.display import display, JSON
from src.ingestion.grants_gov import GrantsGovIngestor
from src.tagging.rule_based import RuleBasedTagger
from src.tagging.ontology import Ontology
from src.extraction.schema import FOARecord

print("✅ Core modules loaded successfully.")

✅ Core modules loaded successfully.


## 1. 🔍 Ingestion & Extraction

We'll start by searching for funding opportunities related to **Artificial Intelligence** on Grants.gov.

In [2]:
ingestor = GrantsGovIngestor()
print(f"Searching for AI opportunities...")

raw_results = ingestor.search("artificial intelligence", max_results=3)
print(f"Found {len(raw_results)} results.\n")

# Show the raw structure of the first result
print("Raw Data Example (Keys):")
print(list(raw_results[0].keys()))

Searching for AI opportunities...
Found 3 results.

Raw Data Example (Keys):
['id', 'number', 'title', 'agencyCode', 'agency', 'openDate', 'closeDate', 'oppStatus', 'docType', 'cfdaList']


## 2. 🏷️ Semantic Tagging

Now we apply our rule-based semantic tagger which uses a **controlled ontology** (`config/ontology.yaml`).

In [3]:
ontology = Ontology()
tagger = RuleBasedTagger(ontology=ontology)

processed_records = []

for raw in raw_results:
    # Standardize fields
    fields = ingestor.extract_fields(raw)
    
    # Apply tags using Title + Description (if available)
    tags = tagger.tag(fields['title'], fields.get('program_description', ""))
    fields['semantic_tags'] = tags
    
    # Create Pydantic record
    record = FOARecord(**fields)
    processed_records.append(record)

print(f"✅ Processed {len(processed_records)} records with semantic tagging.")

✅ Processed 3 records with semantic tagging.


## 3. 📊 Results Visualization

Let's look at what we've extracted in a clean table.

In [4]:
# Convert records to flat dictionaries for display
df = pd.DataFrame([r.to_flat_dict() for r in processed_records])
display(df[['title', 'agency', 'open_date', 'semantic_tags']])

,title,agency,open_date,semantic_tags
0,BJA FY25 Local Law Enforcement Crime Gun Intel...,Bureau of Justice Assistance,2026-02-19,"sponsor_themes/national_security (0.20, rule_b..."
1,Pathways to Enable Secure Open-Source Ecosystems,U.S. National Science Foundation,2026-02-19,
2,Building EPSCoR-State/National Laboratory Part...,Office of Science,2026-02-18,"methods/experimental (0.17, rule_based); spons..."


## 4. 🗄️ Detailed Data Inspection

Evaluators can inspect the full JSON structure for a single record to see the granularity of our extraction.

In [5]:
display(JSON(processed_records[0].model_dump(mode="json")))

<IPython.core.display.JSON object>

---